Name: surahi chandrakant bhor

no: 1262240806

In exp 2 I had taken Topic categorizaton application, So  in this exp I am taking Sentiment Analysis.

In [ ]:
# Install necessary libraries
!pip install nltk scikit-learn gensim tensorflow

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import re
import nltk
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv('/content/drive/MyDrive/IMDB Dataset.csv')

In [3]:
print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample Data:")
display(df.head())
df.isnull().sum()

Dataset Shape: (50000, 2)

Columns: ['review', 'sentiment']

Sample Data:


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


,0
review,0
sentiment,0


In [4]:
def clean_text(text):
    text = re.sub(r'<br />', ' ', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove special characters and numbers
    text = text.lower().split()  # Convert to lowercase and tokenize
    return ' '.join(text)

In [5]:
df['cleaned_review'] = df['review'].apply(clean_text)

In [6]:
def preprocess_labels(label):
    return 1 if label == 'positive' else 0

In [7]:
df['sentiment'] = df['sentiment'].apply(preprocess_labels)

In [8]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2, random_state=42)


In [9]:
# 1. TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

In [10]:
# 2. Word2Vec Embeddings
sentences = [review.split() for review in X_train]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [11]:
def get_w2v_vectors(sentences, model, vector_size=100):
    vectors = []
    for words in sentences:
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            vectors.append(np.mean(word_vectors, axis=0))
        else:
            vectors.append(np.zeros(vector_size))
    return np.array(vectors)

In [12]:
X_train_w2v = get_w2v_vectors(sentences, word2vec_model)
X_test_w2v = get_w2v_vectors([review.split() for review in X_test], word2vec_model)


In [13]:
# 3. Logistic Regression (TF-IDF)
logistic_model = LogisticRegression()
logistic_model.fit(X_train_tfidf, y_train)
y_pred_logistic = logistic_model.predict(X_test_tfidf)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_logistic))
print(classification_report(y_test, y_pred_logistic))

Logistic Regression Accuracy: 0.8943
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      4961
           1       0.89      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



In [22]:
import joblib
# Save the model and vectorizer
joblib.dump(logistic_model, "sentiment_model.pkl")
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [14]:
# 4. Random Forest (TF-IDF)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.851
              precision    recall  f1-score   support

           0       0.84      0.86      0.85      4961
           1       0.86      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



In [15]:
# 5. CNN Model (Word2Vec)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=100)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=100)


In [16]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 100))
for word, i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 100, weights=[embedding_matrix], input_length=100, trainable=False),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_seq, y_train, validation_data=(X_test_seq, y_test), epochs=5, batch_size=64)


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.7214 - loss: 0.5371 - val_accuracy: 0.8306 - val_loss: 0.3732
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8583 - loss: 0.3279 - val_accuracy: 0.8268 - val_loss: 0.3812
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8912 - loss: 0.2609 - val_accuracy: 0.8420 - val_loss: 0.3606
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9139 - loss: 0.2130 - val_accuracy: 0.8430 - val_loss: 0.3676
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9381 - loss: 0.1614 - val_accuracy: 0.8361 - val_loss: 0.4337


In [20]:
cnn_accuracy = model.evaluate(X_test_seq, y_test)[1]
print("CNN Accuracy:", cnn_accuracy)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8348 - loss: 0.4298
CNN Accuracy: 0.8360999822616577


## Conclusion:

In this experiment, we performed sentiment analysis on the IMDB dataset using Logistic Regression, Random Forest, and CNN with TF-IDF and Word2Vec vectorization techniques.

Logistic Regression performed the best with 89.43% accuracy, effectively handling text data with TF-IDF.

Random Forest achieved 85.1% accuracy, but struggled with high-dimensional text features.

CNN obtained 83.6% accuracy, indicating the potential of deep learning but requiring further optimization.

Overall, Logistic Regression was the most effective model, while CNN can be improved using pre-trained embeddings or hyperparameter tuning for better performance.